<h2>MODEL TESTING</h2>

Script ini digunakan hanya untuk mengetes model-model yang telah dibuat. Tahapan dari mengetes suatu model adalah:
1. Load model yang telah ada. (Saat ini, model yang diloat adalah model v1 yang sudah saya buat).
2. Load x_test dan y_test, untuk menjadi data yang dapat dimasukkan ke model untuk melihat performa model. Bagian ini tidak perlu diubah jika model yang diload berubah.
3. Jalankan block terakhir. Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda

In [27]:
import tensorflow as tf
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import math

In [28]:
## Load the model
model = tf.keras.models.load_model('../models/model_v3.keras')

# Show the model architecture
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization (Normalization)   │ (None, 50, 2)          │             5 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 50, 128)        │        67,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 20)             │         2,580 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 10, 2)          │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 603,715 (2.30 MB)

 Trainable params: 201,236 (786.08 KB)

 Non-trainable params: 5 (24.00 B)

 Optimizer params: 402,474 (1.54 MB)

In [29]:
## Load numpy test data
x_test = np.load('../x_test_D.npy')
y_test = np.load('../y_test_D.npy')
mean_variance = np.load('../mean_variance_D.npy')

In [30]:
## IMPORTANT: Set up denormalizer
mean = mean_variance[0]
variance = mean_variance[1]

denormalizer = tf.keras.layers.Normalization(mean=mean, variance=variance, invert=True)

In [45]:
## ONE STEP PREDICTION
# Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda
num = 20

# Predict the next coords of entry "num" using the model
num_of_timesteps = len(x_test[0])
pred_coords = denormalizer(model.predict(x_test[num].reshape(1, num_of_timesteps, 2)))
true_coords = denormalizer(y_test[num])

# Print the true and predicted coordinates
print("Prediction:")
print(pred_coords)

print("True Coordinate:")
print(true_coords)


## Map display of True Point vs Predicted Point
# Get route coordinates
past_lat = []
past_lon = []

for point in x_test[num]:
  past_lat.append(point[0])
  past_lon.append(point[1])

# Get prediction coordinates
pred_lat = []
pred_lon = []

for point in pred_coords[0]:
  pred_lat.append(point[0])
  pred_lon.append(point[1])

# Get true coordinates from y
true_lat = []
true_lon = []

for point in true_coords[0]:
  true_lat.append(point[0])
  true_lon.append(point[1])


# Build the map
fig = go.Figure()

fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=past_lat,
  lon=past_lon,
  name="trip",
  marker={'size': 10}))

fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=pred_lat,
  lon=pred_lon,
  name="Predicted Point",
  marker={'size': 10, 'color': 'green'}))

fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=true_lat,
  lon=true_lon,
  name="True Point",
  marker={'size': 10, 'color': 'yellow'}))

fig.update_layout(mapbox_style="open-street-map",)
fig.update_layout(
  margin={"r":0,"t":0,"l":0,"b":0},
  mapbox=dict(
    zoom=10,
    center=go.layout.mapbox.Center(
            lat=-6.2,
            lon=106.816
        ),
  ))
fig.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Prediction:
tf.Tensor(
[[[ -6.177553  106.79457  ]
  [ -6.1706095 106.79337  ]
  [ -6.1644783 106.79198  ]
  [ -6.1596932 106.790436 ]
  [ -6.1553407 106.78817  ]
  [ -6.151672  106.78598  ]
  [ -6.148948  106.78429  ]
  [ -6.1457644 106.781845 ]
  [ -6.144277  106.77906  ]
  [ -6.1422405 106.777306 ]]], shape=(1, 10, 2), dtype=float32)
True Coordinate:
tf.Tensor(
[[[ -6.181751  106.79412  ]
  [ -6.185277  106.785416 ]
  [ -6.1883273 106.77442  ]
  [ -6.190142  106.76499  ]
  [ -6.1909766 106.75338  ]
  [ -6.1915593 106.74538  ]
  [ -6.191703  106.74232  ]
  [ -6.189238  106.74148  ]
  [ -6.1909666 106.73954  ]
  [ -6.190286  106.734436 ]]], shape=(1, 10, 2), dtype=float32)


In [74]:
## MULTI STEP PREDICTION
# Num dapat diubah-ubah untuk melihat trajectory yang berbeda-beda
num = 1
steps = 12

# Good num = 1

# Predict the next "step" coords of entry "num" using the model
num_of_timesteps = len(x_test[0])
x_data = x_test[num] # Data that will be fed to the model

pred_coords = []
for i in range(steps):
  current_pred = denormalizer(model.predict(x_data.reshape(1, num_of_timesteps, 2))).numpy().tolist()[0]

  pred_coords.append(current_pred)

  x_data = x_data[10:, :]
  
  x_data = np.append(x_data, current_pred, axis=0)

true_coords = denormalizer(y_test[num])

# Print the true and predicted coordinates
print("Prediction:")
print(pred_coords)

print("True Coordinate:")
print(true_coords)


### Map display of True Point vs Predicted Point

# Get route coordinates
past_lat = []
past_lon = []
for point in x_test[num]:
  past_lat.append(point[0])
  past_lon.append(point[1])

# Get prediction coordinates
pred_lat = []
pred_lon = []
for step in pred_coords:
  for point in step:
    pred_lat.append(point[0])
    pred_lon.append(point[1])

# Get true coordinates from y
true_lat = []
true_lon = []

for point in true_coords[0]:
  true_lat.append(point[0])
  true_lon.append(point[1])

# Build the map
fig = go.Figure()

# Plot previous path
fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=past_lat,
  lon=past_lon,
  name="trip",
  marker={'size': 10}))

# Plot predicted path
fig.add_trace(go.Scattermapbox(
  mode="markers+lines",
  lat=pred_lat,
  lon=pred_lon,
  name="Predicted Point",
  marker={'size': 10, 'color': 'green'}))

fig.add_trace(go.Scattermapbox(
  mode="markers",
  lat=true_lat,
  lon=true_lon,
  name="True Point",
  marker={'size': 10, 'color': 'yellow'}))

fig.update_layout(mapbox_style="open-street-map",)
fig.update_layout(
  margin={"r":0,"t":0,"l":0,"b":0},
  mapbox=dict(
    zoom=10,
    center=go.layout.mapbox.Center(
            lat=-6.2,
            lon=106.816
        ),
  ))
fig.show()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Prediction:
[[[-6.153151035308838, 106.79208374023438], [-6.1475653648376465, 106.790283203125], [-6.141993522644043, 106.78782653808594], [-6.137670516967773, 106.78465270996094], [-6.134208679199219, 106.78089141845703], [-6.1314826011657715, 106.7774887084961], [-6.129842281341553, 106.7747573852539], [-6.127762317657471, 106.77189636230469], [-6.127416610717773, 106.76825714111328], [-6.126842975616455, 106.76565551757812]], [[-6.122034072875977, 106.76487731933594], [-6.119673252105713, 106.76136779785156], [-6.117011070251465, 106.7